## Umgebung vorbereiten

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/edadunashvili/ThePrax.git

In [ ]:
cd ThePrax

In [ ]:
!pwd

In [ ]:
!pip install pyprind

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Korpus von etikettierten Daten erstellen

 !!! Den gesuchten Typ bestimmen !!!

In [ ]:
typNum='300'
episode_string_train = "a300_string_train.csv"
episode_roh_train = "a300_roh_train.csv"

Vorhandener gleichnamiger Korpus wird gelöscht 

In [ ]:
import os
if os.path.exists(episode_string_train):
    os.remove(episode_string_train)
else:
    print("Diese Datei existiert nicht")

Im Ordner "Trainingsdaten" nach den entsprechenden Textdateien suchen und in einer rohe Datei zusammentragen

In [ ]:
import glob

def word_to_lex(word):
    ret=(word) 
    return ret

def write_back(words):
    with open(episode_roh_train,"a", encoding='utf-8') as output:
        for word in words:
            #print(word)
            as_lex = word_to_lex(word[0])
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)

def clean(line):
    line = line.replace("\n"," ").strip().lower()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe").replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","").replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a").replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ").replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ").replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','').replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret

with open(episode_roh_train, "w", encoding='utf-8') as output:
    output.write ("quelle,episode,index_string,index_binar\n")
pairs = []


for file in glob.glob("Trainingsdaten/*.txt"):
    
    if typNum in file:
        with open(file, 'r', encoding='utf-8') as episode:
            for line in episode.readlines():
                clean_words = clean(line)
                pairs = pairs + clean_words
write_back(pairs)

Rohdatei endgültig überarbeiten und in eine CSV Datei speichern

In [ ]:
fin = open(episode_roh_train,'r', encoding ='utf-8')
fout = open(episode_string_train, "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"").replace("', '", "','").replace(" '", "'"))                      
fin.close()
fout.close()

Liste von einmaligen Episoden aus der CSV Datei zeigen

In [ ]:
import pandas as pd
df = pd.read_csv(episode_string_train, encoding='utf-8')
from collections import Counter
indexliste=Counter(df.index_string)
print(*indexliste, sep='\n')

## Auswerter

Elementenstruktur definieren: vorherige Episode - gesuchte Episode - folgende Episode

!!! Datei definieren !!!

In [ ]:
import pandas as pd
df = pd.read_csv('a300_string_train.csv', encoding='utf-8')
from collections import Counter

def ep_format(ep_full):
    return (ep_full.split('_'))[0]

def ep_name_format(ep_full):
    sublist = (ep_full.split('_'))[0:2]
    name = ""
    for strg in sublist:
        name += strg+'_'
    return name

def ep_name_vollformat(ep_full):
    sublist = (ep_full.split())
    vollname = ""
    for strg in sublist:
        vollname += strg
    return vollname

def quellenvergleich (df, i1, i2):
#     print(df.quelle[i1],df.quelle[i2])
    return df.quelle[i1]==df.quelle[i2]

def ast(gesep, df):
    ep_tree = {}
    a_liste = []
    z_liste = []  
    df_len = len(df.index_string)
    for i, ep in enumerate(df.index_string):
        if gesep == ep:
             #print(i, ep_full)
            if (i > 0)&(quellenvergleich(df, i, i-1)):
                a = df.index_string[i-1]
            else:
                a = 'eAnfang_a_'
            if (i < df_len - 1):
                if not (quellenvergleich(df, i, i+1)):
                    z = 'eEnde_a_'
                else:        
                    z = df.index_string[i+1]
            else:
                z = 'eEnde_a_'
            a_liste.append(a)
            z_liste.append(z)
    return {gesep: [Counter(a_liste), Counter(z_liste)]}

Alle im Korpus vorhandene Episoden extrahieren

In [ ]:
def alle_aeste(gesep, df):
    episoden_baeume = {}
    
    ep_list = []
    for ep_full in df.index_string:  
        
        ep = ep_format(ep_full)      
        if gesep == ep:          
            
            ep_list.append(ep_full)
    for ep in set(ep_list):
        episoden_baeume.update(ast(ep,df))
    return episoden_baeume

Episoden die gesuchte Episode umlegen und in die Liste in die richrige Reihenfolge darstellen

In [ ]:
def key_val_printer(d):
    for k, v in d.items():
        print(v, k, sep=':')
    #print()
        
def baum_printer(baeume: dict, baum: str):
    key_val_printer (baeume[baum][0])
    
    
    print(80*'-')
    print(sum(baeume[baum][0].values()), baum)
    print(80*'-')
    key_val_printer(baeume[baum][1]) 
    
            
def wald_printer(wald: dict):
    for baum in sorted(wald.keys()):
        baum_printer (wald, baum)
        print(80*'=')

In [ ]:
def key_val_printer(d):
    for k, v in d.items():
        print(v, k, sep=':')
    #print()
#def graph_printer(graph: dict):
    #for 

!!! Die gesuchte Episode bestimmen !!!

In [ ]:
graph = alle_aeste("'e300", df)
graph.update(alle_aeste("'e303", df))
wald_printer(graph)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def cutoff_ast_data(wuerzel, graph, nachbar_liste, cutoff = 0):
    
    vor = graph[wuerzel][0]
    zurueck = graph[wuerzel][1]
    
    nachbarn = vor.copy()
    nachbarn.update(zurueck)
     
    for ast_name, ast_gewicht in sorted(nachbarn.items()):
        if (ast_gewicht >= cutoff):
             nachbar_liste.append([ep_name_format(wuerzel), ep_name_format(ast_name), ast_gewicht])
            
                
def cutoff_ast_volldata(wuerzel, graph, nachbar_liste, cutoff = 0):
    
    vor = graph[wuerzel][0]
    zurueck = graph[wuerzel][1]
    
    nachbarn = vor.copy()
    nachbarn.update(zurueck)
     
    for ast_name, ast_gewicht in sorted(nachbarn.items()):
        if (ast_gewicht >= cutoff):
             nachbar_liste.append([ep_name_vollformat(wuerzel), ep_name_vollformat(ast_name), ast_gewicht])
                
                
def cutoff_graph_data(graph, cutoff = 0):
    nachbar_liste = []
    for i, (k, v) in enumerate(sorted(graph.items())):
        cutoff_ast_data(k, graph, nachbar_liste, cutoff)
    neue_nachbar_liste = [[i, nachbar] for i, nachbar in enumerate(nachbar_liste)]
    return neue_nachbar_liste

def cutoff_graph_volldata(graph, cutoff = 0):
    nachbar_liste = []
    for i, (k, v) in enumerate(sorted(graph.items())):
        cutoff_ast_volldata(k, graph, nachbar_liste, cutoff)
    neue_nachbar_liste = [[i, nachbar] for i, nachbar in enumerate(nachbar_liste)]
    return neue_nachbar_liste

def interactive_graph_data(graph_data, loesch_index, gew_dict):
    
    for k, v in gew_dict.items():
        graph_data[k][1][2] = v
    neue_nachbar_liste = [[i, nachbar[1]] for i, nachbar in enumerate(graph_data) if i not in loesch_index]

    return neue_nachbar_liste

def graph_bauer(graph_data):
    G = nx.Graph()
    for el in graph_data:
        n1 = el[1][0]
        n2 = el[1][1]
        w = el[1][2]
        G.add_edge(n1, n2,weight=w)
    return G

def show_graph(G):
    fh = open("edgelist.utf-8", "wb")
    nx.write_multiline_adjlist(G, fh, delimiter="\t", encoding="utf-8")

    # read and store in UTF-8

    fh = open("edgelist.utf-8", "rb")
    H = nx.read_multiline_adjlist(fh, delimiter="\t", encoding="utf-8")

    pos = nx.spring_layout(G)
    for p in pos:  # raise text positions
        pos[p][0] += 0.07
    nx.draw(G, pos, font_size=12, with_labels=True)

    nx.draw_networkx_labels(G, pos, font_size=12, with_labels=True)
    plt.show()

In [ ]:
heufigkeit=4
auto_graph=cutoff_graph_data(graph, heufigkeit)
#auto_graph

In [ ]:
interactive_data = interactive_graph_data(auto_graph,[],{})
#print(*interactive_data, sep='\n')
G = graph_bauer(interactive_data)
show_graph(G)

mit_werte=cutoff_graph_volldata(graph, heufigkeit)
mit_werte
